In [21]:
import Modules.SQcircuit_extensions as sq_ext
import Modules.figures as figs
import SQcircuit as sq
import numpy as np
import matplotlib.pyplot as plt
import importlib
import qutip as qt

plt.rcParams['text.usetex'] = False
importlib.reload(sq_ext)
importlib.reload(sq)
importlib.reload(figs)
np.set_printoptions(linewidth=200, formatter={'float': '{:.5f}'.format})

<Figure size 640x480 with 0 Axes>

In [22]:
GHz = 1e9
fF  = 1e-15
nH  = 1e-9

In [23]:
def resonator_C_resonator(Cc, C_val, L_val):
    Cc = sq.Capacitor(Cc, 'fF')
    C = sq.Capacitor(C_val,   'fF')
    L = sq.Inductor (L_val,   'nH')
    elements = {(0, 1): [L, C],
                (0, 2): [L, C],
                (1, 2): [Cc], }
    return sq.Circuit(elements)

def resonator(Cc, C_val, L_val):
    if Cc == 0:
        C = sq.Capacitor(C_val, 'fF')
        L = sq.Inductor(L_val,  'nH')
        elements = {(0, 1): [L, C], }
        return sq.Circuit(elements)
    else:
        C = sq.Capacitor(1/(1/C_val+1/Cc), 'fF')
        L = sq.Inductor (L_val, 'nH')
        elements = {(0, 1): [L, C], }
    return sq.Circuit(elements)
    

In [24]:
n_eig = 8
trunc_num = 25

In [4]:
C_val = 2
L_val = 1
Cc = 51

In [5]:
coupled_res = resonator_C_resonator(Cc=Cc, C_val=C_val, L_val=L_val)
res = resonator(Cc=Cc, C_val=C_val, L_val=L_val)

coupled_res.set_trunc_nums([trunc_num, trunc_num])
res.set_trunc_nums([trunc_num  ])

_ = coupled_res.diag(n_eig)
_ = res.diag(n_eig)

In [6]:
Q = res.charge_op(0)
Φ = res.flux_op(0)

In [7]:
C_mat = np.array([[C_val, -Cc],
                  [-Cc, C_val]])
C_inv = np.linalg.inv(C_mat)
C_inv

array([[0.13333, 0.06667],
       [0.06667, 0.13333]])

In [8]:

C1_tilde = C_inv[0,0]**-1
C2_tilde = C_inv[1,1]**-1
C_coup = C_inv[0,1]**-1 /3
C_coup, Cc

(5.0, 5)

In [9]:
C_coup = Cc

In [10]:
H_res   = res.hamiltonian()
I = qt.identity(H_res.shape[0])

H_coup_res = qt.tensor(H_res, I) + qt.tensor(I, H_res) -  (0.5 / (C_coup*fF)) * qt.tensor(Q, Q)

In [14]:
E = sq_ext.diag(H_coup_res,n_eig=n_eig, out='GHz')[0]
E -= E[0]
E/2

array([0.00000, 35.58813, 50.32921, 71.17625, 85.91734, 100.65842, 106.76438, 121.50547])

In [15]:
E = coupled_res.efreqs - coupled_res.efreqs[0]
E

array([0.00000, 35.58813, 50.32921, 71.17625, 85.91734, 100.65842, 106.76438, 121.50547])